# env setup

## import

In [ ]:
import json
import os
import datetime as dt

def save_checkpoint():
    os.makedirs("./self_trpg_ckpt", exist_ok=True)
    fn = dt.datetime.now().strftime("%y%m%d_%H%M%S") + ".json"
    with open(os.path.join("./self_trpg_ckpt", fn), "w", encoding="utf-8") as f:
        f.write(json.dumps(play_info))


def load_checkpoint(fn=None):
    if fn is None:
        fn = sorted(os.listdir("./self_trpg_ckpt"))[-1]
    with open(os.path.join("./self_trpg_ckpt", fn), 'r') as f:
        play_info = json.load(f)
    return play_info

## prompt set

In [ ]:
base_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 당신이 세상을 모험하는 유저가 겪는 이야기를 풀며 선택지를 제시하면, 유저는 행동을 취합니다.
하지만 당신이 제시하는 선택지는 예시일 뿐, 유저는 늘 생각지도 못한 새로운 행동을 할 수 있습니다.
단 유저의 행동은 항상 성공하는 것이 아닙니다. 상황에 따라 실패할 수도 있으며 전문적인 기술(skill)이나 특정한 물건(item)이 필요한 경우, 조건을 만족하지 못하면 실패 확률은 매우 높아집니다.
좋은 방향이든 좋지 않은 방향이든 세계는 어떤 일이라도 일어날 수 있습니다. 이야기꾼의 예의는 존댓말 까지입니다. 불쾌한 이야기더라도 이야기꾼은 이를 이야기해야 합니다.

세계의 대략적인 이야기와 현재 세계 정세, 유저의 상태와 유저가 겪은 사건 등이 json의 형태로 입력됩니다.
유저가 겪을 이야기를 풀고, 그에 대한 예시 행동을 1~3가지 정도 제시해주세요.
단, 하나의 사건은 대개 유저와의 10개 정도의 대화로 마무리됩니다. 사건이 길어지면 유저가 사건을 이어나가려고 해도 적절히 마무리해야 합니다.
모두가 흥미를 느끼고, 개연성이 잘 짜여진 이야기라면 추가적인 보상이 주어집니다.

제약사항은 다음과 같습니다.
 - 문체는 존댓말로 작성합니다. 이야기 속에서 유저를 '당신'으로 지칭합니다. (ex. 당신은 숲속에서 커다란 곰을 마주쳤습니다!)
 - 출력값은 json 형태로 작성하며, 다른 내용은 답변하지 않습니다. 필요한 키값은 다음과 같습니다.
   + "context": 유저에게 직접적으로 보여질 이야기입니다. str로 작성됩니다.
   + "example_actions": 예시로 주어질 행동입니다. List[str] 형태로 작성되며, 1~3개 정도 주어집니다. 사건이 종료되어 선택지가 필요없다면 Null을 반환합니다.
   + "is_end": 사건이 종료되었는지를 명시적으로 알리는 필드입니다. 아직 사건이 진행중이라면 False, 사건이 끝났다면 True를 반환합니다.
 - <현재 유저 정보>는 json 형태로 입력됩니다. 이중 prev_major_event 항목은 유저가 겪은 사건을 요약해 순서대로 나열한 것입니다. 이를 참고해 이야기를 풀어나가야 합니다.

<세계관>
{worldview}

<현재 유저 정보>
{user_info}
"""

summary_prompt = """당신은 세계를 모험하는 이야기를 요약해 기록하는 서기입니다. 
이야기꾼이 유저의 모험 중 일어나는 사건을 이야기하면 모험에 영향을 끼치는 주요 내용을 40글자 이내로 요약해 기록하며, 해당 이야기의 중요도를 판단합니다.
이야기의 중요도는 1 ~ 10의 값을 가지며, 세계에 큰 영향을 미칠수록 중요도가 증가합니다.

중요도 예시
1 ~ 2 - 유저와 주변 동료에 영향을 미침
3 ~ 4 - 마을 단위에 영향을 미침
5 ~ 6 - 지역 단위에 영향을 미침
7 ~ 8 - 국가 단위에 영향을 미침
9 ~ 10 - 세계 전역에 영향을 미침

결과물은 json 형태로 작성되며 다른 내용은 답변하지 않습니다. 답변 예시는 다음과 같습니다.
{{"summary": str, "importance": int}}

모두가 한눈에 읽고 이해할 수 있도록 요약해야 하며, 성공적으로 작업을 완료할 시 인센티브가 주어집니다.

<이전 사건>
{story_context}
"""

state_update_prompt = """당신은 세계를 모험하는 이야기를 주시하며 유저의 상태를 관리하는 관리자입니다. 
어떤 사건의 경위를 읽고 현재 유저의 상태와 비교하며 유저가 받은 피해, 얻은 아이템 등을 정리하여 구조화해 작성합니다.
결과물은 json 형태로 작성하며, 다른 형태로 작성하거나 설명 태그 등은 붙이지 않습니다. 당신이 작성할 필드는 다음과 같습니다.
 - "current_location": 유저의 현재 위치입니다. 자세할수록 더 좋습니다. str 형태로 작성됩니다.
 - "hp": 유저의 현재 체력입니다. +-int 형태로 작성됩니다. (최초 기본 체력은 100입니다.)
 - "mental": 유저의 현재 정신력입니다. +-int 형태로 작성됩니다. (최초 기본 정신력은 100입니다.)
 - "max_hp": 유저의 최대 체력입니다. +-int 형태로 작성됩니다.
 - "max_mental": 유저의 최대 정신력입니다. +-int 형태로 작성됩니다.
 - "skills": 보유한 기술입니다. 타인과 구분할 수 있는 유의미한 기술만 기록됩니다. list[str] 형태로 작성됩니다.
 - "items": 보유한 아이템입니다. list[str] 형태로 작성됩니다.
 - "companion": 모험에 함께하는 동료입니다. 굳이 사람이 아니더라도 함께할 수 있습니다. list[dict] 형태로 작성하며, 양식은 다음과 같습니다.
   - "name": 동료의 이름입니다.
   - "info": 동료의 간단한 설명입니다. 종족이나 직업, 성격 등을 최대한 간략하고 명료하게 작성합니다.

유저의 최대 체력과 정신력은 각각 100입니다. 납득할 수 있는 수준으로 책정해야 하며, 모두가 납득할만한 정확한 수치를 책정시 인센티브가 주어집니다.

<유저 정보>
{user_info}

<이전 사건>
{story_context}
"""

theme_gen_prompt = """당신은 유저가 모험할 세계의 기본적인 세계관을 정하는 각본가입니다. 하나의 거대한 테마와 유저가 원하는 키워드를 입력받아 세계관을 작성합니다.
작성하는 문체는 간결하고 정보 전달만을 목적으로 하며, 간단한 지리 설명이나 주요한 설명 등이 필요합니다. 

작성 예시는 다음과 같습니다.
주요 테마: 퓨전 판타지
키워드: 단일 대륙, 중세 판타지, 무협지, 몬스터
생성결과:
아르카나는 한개의 큰 대륙으로 이루어진 세계이자, 하나뿐인 대륙의 이름이다. 
대륙 중앙엔 동대륙과 서대륙을 나누는 거대한 산맥이 존재하며, 약간의 교류를 이어나가고 있다.
서대륙은 흔히 말하는 판타지 세상이다. 기사와 마법사, 신관이 존재하며 각각 오러, 마력, 신성력을 다룬다.
서대륙은 중앙의 '프로스트 제국'을 중심으로 동서남북에 각각 '프레이야 왕국', '바르글룸 왕국', '호슬로 공국', '이슈트반 연합국'으로 이루어져 있다.
반면 동대륙은 흔히 말하는 무협 세상이다. 내공을 이용한 무공을 사용하는 무인, 도력을 사용하는 도사, 사술 등을 사용한다.
동대륙은 서대륙과 달리 전역을 '무림'이라 칭한다. 무림 내부엔 문파나 가문으로 뭉쳐있으며 별개의 '마교'가 존재한다.
동대륙의 문파는 대중에게 알려진 무협지와 동일하게 남궁세가, 제갈세가, 무당파, 소림사 등으로 이루어져 있다.
각 대륙의 기술은 중세 수준으로, 아직 화약을 발견하기 이전의 시대이다.
대륙 전역엔 괴물이 나타난다. 어린 아이도 물리칠 수 있는 약한 개체부터 나라를 멸망시킬 수준의 강한 괴물까지 다양하다.
서대륙에선 몬스터, 동대륙에선 요괴라 부르며, 용병이나 기사, 무인 등 전투인력은 몬스터 처리가 주 업무이다.

입력된 값
주요 테마: {theme}
키워드: {keywords}
생성결과:
"""


start_set_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 이야기를 시작하기 전, 전체적인 스토리를 보고 주인공을 정하려고 합니다.
세계관의 무대 내에서 활동할 주인공을 정하고, 주인공의 현재 위치, 이야기의 시작이 될 내용을 json 형태로 출력합니다.
주인공은 세계관의 중심적인 인물일지도, 전혀 중요하지 않은 엑스트라에 불과할 수도 있습니다. 이야기의 시작은 40글자 이내로 간결하게 작성합니다.


<예시>
{{
    "user_role": "바르글룸의 왕국의 목장 마을 소년"
    "current_location": "바르글룸 왕국, 목장 마을",
    "start_event": "바르글룸의 작은 목장 마을 소년인 유저가 모험을 결심함"
}}

<이야기의 세계관>
{worldview}

! 주인공의 성별은 {sex}입니다.
"""

## request setup

In [ ]:
import openai
import torch
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline

model_type = "tgi"


if model_type == "openai":
    openai



model_names = [
    "davidkim205/komt-mistral-7b-v1",  # 무난, 성능 약간 딸림. multi turn 애매할지도
    "wkshin89/mistral-7b-instruct-ko-test-v0.1",  # 비슷함
    "beomi/gemma-ko-7b",  # tgi 기준 4bit oom. transformers는 모름
    "lemon-mint/gemma-ko-7b-instruct-v0.71",  # 위와 동일
    "Qwen/Qwen2-7B-Instruct",  # tgi 기준, greedy 아니면 병신
    "beomi/Llama-3-KoEn-8B-Instruct-preview",  # tgi 기준, eos 인식이 안됨. transformer에선 잘하는듯? instruction 테스트는 다시 해봐야함
    "beomi/Llama-3-Open-Ko-8B",  # 위와 동일한 증상. 성능도 비슷하지만 instruction 아니라 애매함
    "wanotai/Kwen2-7B-Instruct-Preview",  # 나쁘지 않은데 성능은 좀 애매한듯. gpt4가 사기긴 함... preview라 좀 봐야함
    "spow12/Ko-Qwen2-7B-Instruct",  # 성능이 영 아닌듯. 다른 친구들은 잘 만든다
]
model_name = 'wanotai/Kwen2-7B-Instruct-Preview'

headers = {
    "Content-Type": "application/json",
}


if model_type in ["tgi", "transformers"]:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if model_type == "transformers":
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_4bit=True)
        pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer)

In [ ]:
def request_query(messages, model="openai", args={}):
    if model == "openai":
        completion = openai.ChatCompletion.create(
            # engine="illunex-ai-gpt3", 
            engine="illunex-ai-gpt4-prompt",
            messages=messages,
        )
        result = completion.choices[0]['message']['content']
        
    elif model in ['tgi', 'transformers']:
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
        if model == "tgi":
            response = requests.post(
                'http://127.0.0.1:8080/generate', 
                headers=headers, 
                json={'inputs': text, 'parameters': args},
            )
            result = response.json()['generated_text']

        
        elif model == "transformers":
            result = pipe(text, **args)[0]['generated_text']
            
    else:
        raise KeyError("`model` is Literal['openai', 'tgi', 'transformers']")

    return result

# story setup

## set theme, worldview

In [ ]:
theme_gen_prompt = """당신은 유저가 모험할 세계의 기본적인 세계관을 정하는 각본가입니다. 하나의 거대한 테마와 유저가 원하는 키워드를 입력받아 세계관을 작성합니다.
작성하는 문체는 간결하고 정보 전달만을 목적으로 하며, 간단한 지리 설명이나 주요한 설명 등이 필요합니다.
만약 입력되는 값 중 실제 지명 등이 등장한다면 현실세계에 새로운 설정을 덮어씌웁니다. 
주어진 테마를 벗어나지 않는 선에서 키워드들을 잘 배치해 사람들이 흥미를 느낄만한 세계관을 작성해주세요.
"""
# 작성 예시는 다음과 같습니다.
# 주요 테마: 퓨전 판타지
# 키워드: 단일 대륙, 중세 판타지, 무협지, 몬스터
# 생성결과:
# 아르카나는 한개의 큰 대륙으로 이루어진 세계이자, 하나뿐인 대륙의 이름이다. 
# 대륙 중앙엔 동대륙과 서대륙을 나누는 거대한 산맥이 존재하며, 약간의 교류를 이어나가고 있다.
# 서대륙은 흔히 말하는 판타지 세상이다. 기사와 마법사, 신관이 존재하며 각각 오러, 마력, 신성력을 다룬다.
# 서대륙은 중앙의 '프로스트 제국'을 중심으로 동서남북에 각각 '프레이야 왕국', '바르글룸 왕국', '호슬로 공국', '이슈트반 연합국'으로 이루어져 있다.
# 반면 동대륙은 흔히 말하는 무협 세상이다. 내공을 이용한 무공을 사용하는 무인, 도력을 사용하는 도사, 사술 등을 사용한다.
# 동대륙은 서대륙과 달리 전역을 '무림'이라 칭한다. 무림 내부엔 문파나 가문으로 뭉쳐있으며 별개의 '마교'가 존재한다.
# 동대륙의 문파는 대중에게 알려진 무협지와 동일하게 남궁세가, 제갈세가, 무당파, 소림사 등으로 이루어져 있다.
# 각 대륙의 기술은 중세 수준으로, 아직 화약을 발견하기 이전의 시대이다.
# 대륙 전역엔 괴물이 나타난다. 어린 아이도 물리칠 수 있는 약한 개체부터 나라를 멸망시킬 수준의 강한 괴물까지 다양하다.
# 서대륙에선 몬스터, 동대륙에선 요괴라 부르며, 용병이나 기사, 무인 등 전투인력은 몬스터 처리가 주 업무이다.
# """


asdf = """다음과 같은 주요 테마와 키워드로 세계관을 작성하십시오.
주요 테마: {theme}
키워드: {keywords}"""

# messages = [
#     {"role": "system", "content": theme_gen_prompt},
#     {"role": "user", "content": asdf.format(theme=input_theme, keywords=keywords)},
# ]


# text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt').to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# args = {
#     "max_new_tokens": 1024,
#     "eos_token_id": terminators,
#     "do_sample": True,
#     "temperature": 1,
#     "top_p": 0.9,
#     # "return_full_text": False,
# }
# r = model.generate(text, **args)[0]
# print(tokenizer.decode(r.tolist()))

In [ ]:
messages = [
    {"role": "system", "content": "trpg 게임을 주관하는 게임 마스터로서 '좀비 아포칼립스'라는 주제로 게임을 진행하세요. 상황을 안내하면 유저가 답변하여 행동할 것입니다."},
    # {"role": "user", "content": "trpg가 뭐야?"},
]

def call(messages):
    return request_query(
        messages, 
        model_type, 
        {
            "max_new_tokens": 1024,
            "eos_token_id": [
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
            "do_sample": True,
            "temperature": 1,
            "top_p": 0.9,
            "return_full_text": False,
        }
    )


while True:
    if messages[-1]['role'] != "assistant":
        r = call(messages)
        messages.append({"role": "assistant", "content": r})
        
    print('\n' + r + '\n')
    messages.append({"role": "user", "content": input()})

In [ ]:
load_ckpt = False

if load_ckpt:
    play_info = load_checkpoint()
else:
    # theme = [
    #     "좀비 아포칼립스",
    #     "현대 판타지",
    #     "중세 판타지",
    #     "무협지",
    #     "현대 일상",
    #     "퓨전 판타지",
    # ]
    # for i, t in enumerate(theme):
    #     print(f"{i + 1}. {t}")
    
    # input_theme = input("테마 입력: ")
    # keywords = input("키워드 입력: ")
    
    messages = [
        {"role": "system", "content": theme_gen_prompt},
        {"role": "user", "content": asdf.format(theme=input_theme, keywords=keywords)},
    ]
    worldview = request_query(
        messages, 
        model_type, 
        {
            "max_new_tokens": 512,
            "eos_token_id": [
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
            "do_sample": True,
            "temperature": 1,
            "top_p": 0.9,
            "return_full_text": False,
        }
    )
    print("\n\n" + worldview)

    play_info = {
        "main_theme": input_theme,
        "keywords": keywords,
        "worldview": worldview,
        "user_role": None,
        "user_info": {
            "user_sex": None,
            "current_location": None,
            "prev_major_event": None,
            "hp": 100,
            "mental": 100,
            "max_hp": 100,
            "max_mental": 100,
            "skills": [],
            "items": [],
            "companion": [],
        }
    }

In [ ]:
keywords

## set staring point

In [ ]:
messages= [
    {
        "role": "system", 
        "content": start_set_prompt.format(
            worldview=play_info['worldview'],
            sex=sex,
        )
    }
]

r = request_query(
    messages,
    model_type, 
        {
            "max_new_tokens": 512,
            "eos_token_id": [
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
            "do_sample": True,
            "temperature": 1,
            "top_p": 0.8,
            "return_full_text": False,
        }
)
print(r)

In [ ]:
# sex = input("유저의 성별['남성'/여성]: ")

# def default_input(text, default):
#     x = input(text)
#     return default if x.strip() == "" else x

# if default_input("유저 정보를 직접 입력하시겠습니까? [Y/n]: ", "y").lower() == "y":
#     while True:
#         user = input("유저는 어떤 인물인가요?: ")
#         loc = input("유저의 현재 위치는?: ")
#         start = input("이야기를 시작할 때의 상황을 30글자 미만으로 작성해주세요.: ")
#         if default_input("입력한 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
#             break

#     play_info["user_role"] = user
#     play_info["user_info"]["user_sex"] = sex
#     play_info["user_info"]["current_location"] = loc
#     play_info["user_info"]["prev_major_event"] = [start]

# else:
#     while True:
#         completion = openai.ChatCompletion.create(
#             # engine="illunex-ai-gpt3", 
#             engine="illunex-ai-gpt4-prompt",
#             messages = [
#                 {
#                     "role": "system", 
#                     "content": start_set_prompt.format(
#                         worldview=play_info['worldview'],
#                         sex=sex,
#                     )
#                 }
#             ],
#         )
#         start_info = eval(completion.choices[0]['message']['content'])

#         print("\n유저는 '" + start_info['user_role'] + "' 입니다.")
#         print("유저의 현재 위치는 '" + start_info['current_location'] + "' 입니다.")
#         print("이야기를 시작할 때의 시점은 '" + start_info['start_event'] + "' 입니다.")

    
#         if default_input("해당 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
#             break
            
#     play_info["user_role"] = start_info['user_role']
#     play_info["user_info"]["user_sex"] = sex
#     play_info["user_info"]["current_location"] = start_info['current_location']
#     play_info["user_info"]["prev_major_event"] = [start_info['start_event']]

# play

In [ ]:
prompt = base_prompt.format(worldview=worldview, user_info=play_info['user_info'])
# print(prompt)
messages = [
    {"role": "system", "content": prompt}
]

while True:
    # completion = openai.ChatCompletion.create(
    #     # engine="illunex-ai-gpt3", 
    #     engine="illunex-ai-gpt4-prompt",
    #     messages=messages,
    # )
    # result = completion.choices[0]['message']['content']
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    
    
    try:
        result = eval(result)
    except:
        try:
            result = json.loads(result)
        except:
            print(" == regenerate == ")
            continue
            
    messages.append({"role": "assistant", "content": str(result)})
    
    
    print(result['context'])

    if result['is_end']:
        break
    
    print('\n<행동 예시>')
    for i, action in enumerate(result['example_actions']):
        print(f"{i + 1}. {action}")

    user_action = input("\n나는... ")
    messages.append({"role": "user", "content": str({"action": user_action})})
    print()

## after processing

In [ ]:
story_context = ""
for m in messages[1:]:
    content = eval(m['content'])
    if m['role'] == "assistant":
        story_context += "이야기꾼: " + content['context'] + "\n\n"
    elif m['role'] == "user":
        story_context += "유저: " + content['action'] + "\n\n"

In [ ]:
prompt = summary_prompt.format(story_context=story_context)
completion = openai.ChatCompletion.create(
    # engine="illunex-ai-gpt3", 
    engine="illunex-ai-gpt4-prompt",
    messages=[{"role": "system", "content": prompt}],
)
result = completion.choices[0]['message']['content']
result = eval(result)

play_info['user_info']['prev_major_event'].append(result['summary'])

play_info['user_info']

In [ ]:
prompt = state_update_prompt.format(user_info=play_info['user_info'], story_context=story_context)
completion = openai.ChatCompletion.create(
    # engine="illunex-ai-gpt3", 
    engine="illunex-ai-gpt4-prompt",
    messages=[{"role": "system", "content": prompt}],
)
result = completion.choices[0]['message']['content']
result = eval(result)


play_info['user_info']['current_location'] = result['current_location']
play_info['user_info']['max_hp'] = result['max_hp']
play_info['user_info']['max_mental'] = result['max_mental']
play_info['user_info']['hp'] = min(result['hp'], play_info['user_info']['max_hp'])
play_info['user_info']['mental'] = min(result['mental'], play_info['user_info']['max_mental'])
play_info['user_info']['skills'] = result['skills']
play_info['user_info']['items'] = result['items']
play_info['user_info']['companion'] = result['companion']

play_info['user_info']

In [ ]:
save_checkpoint()